Nouveaux concepts : 
-

- Utiliser les mots les tokens en entrée du réseau, en apprenant un embedding, comme ça on rentre tout d'un coup (toutes les infos)

- PPO + LSTM/Transformer comme nouvelle architecture, Q-learning c'est pas ouf car ça nécessite de connaitre tous les states


In [1]:
#1 on se place 
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('../..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git


In [2]:
from poke_env.player.random_player import RandomPlayer
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
import json
from poke_env.environment.pokemon import Pokemon
import poke_env
import pandas as pd
import numpy as np
from poke_env.data import GenData
from poke_env.environment.abstract_battle import AbstractBattle
from poke_env.player.random_player import RandomPlayer
from stable_baselines3.common.env_util import make_vec_env
from IPython.display import display
import ipywidgets as widgets
from poke_env.player import Gen4EnvSinglePlayer
from poke_env.data import GenData
import torch
from gymnasium.spaces import Space, Box, Discrete

In [3]:
p = Pokemon(gen=4,species= '')
print(p.base_stats)
print(p.ability)

{'atk': 92, 'def': 75, 'hp': 90, 'spa': 92, 'spd': 85, 'spe': 60}
snowwarning


Vecteur obs :
-


battle.all_active_pokemons : renvoie les deux pokémons sur le terrain

battle.current_observation : renvoie un sacré truc (détaillé après)

-> Contient les PVs et status de chaque pokémon

Clés de battle.current_observation :
['side_conditions', 'opponent_side_conditions', 'weather', 'fields', 'active_pokemon', 'opponent_active_pokemon', 'team', 'opponent_team', 'events']

events a en majorité rien

In [51]:
from math import floor
#active_pokemon et opponent_pokemon
def ObservedPokemon_to_list(pokemon,is_active_pokemon,is_opponent_team) -> list: 
    # Nom du Pokémon
    species = getattr(pokemon, "species", "UNKNOWN")
    p = Pokemon(gen=4,species= species)


    # Fraction de vie (float entre 0 et 1)
    hp = getattr(pokemon, "current_hp_fraction", 0.0) or 0.0

    # Types (liste de strings ou [])
    types = [p.type_1.name.lower() if p.type_1 else None, p.type_2.name.lower() if p.type_2 else None]
    if p.type_1 and p.type_2 :
        types = sorted([t for t in types])
    type_str = types[0] if types[1] is None else f"{types[0]}_{types[1]}"

    # Statut (string ou "NONE")
    status = getattr(pokemon, "status", None)
    status_str = status.name if status else "NONE"
    status_to_id = {
     'NONE': 0,
    'par': 1,     # paralysie
    'slp': 2,     # sommeil
    'brn': 3,     # brûlure
    'frz': 4,     # gel
    'psn': 5,     # poison
    'tox': 6      # poison grave
        }
    status_id = status_to_id[status_str] 

    #Boosts
    if(is_active_pokemon == 1) :

        # Boosts (dict ou [0]*7)
        boosts = getattr(pokemon, "boosts", None)
        boosts_list = list(boosts.values()) if boosts else [0] * 7

    # Stats (dict ou [0]*6)
    if is_opponent_team == 0 : #Si c'est pas l'équipe adverse :

        stats = getattr(pokemon, "stats", None)
        stat_list = list(stats.values()) if stats else [0] * 6
        stats_list = [ np.float32(x / 504) for x in stat_list ]

    else : #si c'est l'équipe adverse, on fait autrement
        level = p.level if hasattr(p, "level") and p.level else 80
        hp_stat = floor(((2 * p.base_stats["hp"] + 31 + 84 // 4) * level) / 100) + level + 10
        if hp == None : 
            hp = 1.0
            #On rajoute un quasi-normalize, pour aller autour de 1 sans que ça soit trop petit, donc on prend pas l'exemple leuphorie 714 stats en PV avec lvl 100, nature, EVs
        atk = floor((2 * p.base_stats["atk"] + 31 + 84 // 4) * level / 100 + 5) 
        def_ = floor((2 * p.base_stats["def"] + 31 + 84 // 4) * level / 100 + 5) 
        spa = floor((2 * p.base_stats["spa"] + 31 + 84 // 4) * level / 100 + 5) 
        spd = floor((2 * p.base_stats["spd"] + 31 + 84 // 4) * level / 100 + 5) 
        spe = floor((2 * p.base_stats["spe"] + 31 + 84 // 4) * level / 100 + 5) 
        
        stats_list = [hp_stat,np.float32(atk/504),np.float32(def_/504),np.float32(spa/504),np.float32(spd/504),np.float32(spe/504)]
    #moovepool
    moves_dict = getattr(pokemon, "moves", None)
    move_names = list(moves_dict.keys()) if moves_dict else []

    # Pour avoir toujours 4 valeurs :
    while len(move_names) < 4:
        move_names.append("None")

    ability = p.ability
    if(is_active_pokemon == 1) :

        return [
            species,
            hp,
            ability,
            type_str,
            status_id,
            *boosts_list,
            *stats_list,
            *move_names[:4]
        ]
    else : 
        return [
            species,
            hp,
            ability,
            type_str,
            status_id,
            *stats_list,
            *move_names[:4]
        ]

In [5]:
#my_team_switches avec les stats, mooves, %hp, status
from itertools import chain

def get_my_switches_with_all_infos(switches,dict) : 

    # Dictionnaire contenant les ObservedPokemon (via battle.current_observation.__dict__["team"])
    team_obs_dict = dict["team"]

    enriched_switches = []

    for poke in switches:
        # Trouver l'ObservedPokemon correspondant en comparant le species
        matched_obs = next(
            (obs for obs in team_obs_dict.values()
             if hasattr(obs, "species") and obs.species == poke.species),
            None
        )

        if matched_obs:
            enriched_switches += ObservedPokemon_to_list(matched_obs,is_active_pokemon=0,is_opponent_team=0)
        else:
            # Fallback si non trouvé
            enriched_switches += ["UNKNOWN", 0.0, "NONE"] + [0]*7 + [0]*6

    return enriched_switches
    
def get_opponent_team_with_all_infos(dict,battle) : 

    # Dictionnaire contenant les ObservedPokemon (via battle.current_observation.__dict__["team"])
    team_obs_dict = dict["opponent_team"]
    #enlever le pokemon actif adverse
    key_to_remove = next((k for k, v in team_obs_dict.items() if v == battle.opponent_active_pokemon), None)
    if key_to_remove:
            del team_obs_dict[key_to_remove]

    opponent_team_list = []
    for i in team_obs_dict.values() :
        opponent_team_list += ObservedPokemon_to_list(i,is_active_pokemon=0,is_opponent_team=1)


    return opponent_team_list

In [6]:
####Tests embedding ########

class adapted_obs :
    def __init__(self,battle) :
        self.dict_observation = battle.current_observation.__dict__ 
        #et stocker l'état précédent ? Pour l'instant on passe
        self.side_conditions = list(self.dict_observation["side_conditions"].values())

        self.opponent_side_conditions = list(self.dict_observation["opponent_side_conditions"].values())

        self.weather = list(self.dict_observation["weather"].values())

        self.active_pokemon = ObservedPokemon_to_list(self.dict_observation["active_pokemon"],is_active_pokemon=1,is_opponent_team=0)
        self.my_team_switches = get_my_switches_with_all_infos(battle.available_switches,self.dict_observation)
        self.opponent_active_pokemon = ObservedPokemon_to_list(self.dict_observation["opponent_active_pokemon"],is_active_pokemon=1,is_opponent_team=1) #à 1 car on veut savoir si il est boosté
        
        #Equipe adverse sans le pokemon actif 
        self.opponent_team = get_opponent_team_with_all_infos(self.dict_observation,battle)
        #Fin équipe adverse 
        self.output =   ["side_conditions"] + self.side_conditions + ["opponent_side_conditions"] + self.opponent_side_conditions + ["weather"] + self.weather + ["active_pokemon"] + self.active_pokemon + ["my_team_switches"] + self.my_team_switches + ["opponent_active_pokemon"] + self.opponent_active_pokemon + ["opponent_team"] + self.opponent_team

    #def update(self, battle) : #Par la suite on ne renverra plus directement ces valeurs, mais on passera par des embeddings, et là ça pourra être intéressant de ne modifier que quand c'est nécessaire 
    #    new_obs = battle.current_observation.__dict__

        # Check & update side_conditions
    #    new_side_conditions = list(new_obs["side_conditions"].values())
    #    if new_side_conditions != self.side_conditions:
    #        self.side_conditions = new_side_conditions

        # Check & update opponent_side_conditions
    #    new_opponent_side_conditions = list(new_obs["opponent_side_conditions"].values())
    #    if new_opponent_side_conditions != self.opponent_side_conditions:
    #        self.opponent_side_conditions = new_opponent_side_conditions

        # Check & update weather
    #    new_weather = list(new_obs["weather"].values())
    #    if new_weather != self.weather:
    #        self.weather = new_weather

        # Check & update active_pokemon
    #    new_active = ObservedPokemon_to_list(new_obs["active_pokemon"], is_active_pokemon=1, is_opponent_team=0)
    #    if new_active != self.active_pokemon:
    #        self.active_pokemon = new_active

        # Update my_team_switches
    #    new_switches = get_my_switches_with_all_infos(battle.available_switches, new_obs)
    #    if new_switches != self.my_team_switches:
    #        self.my_team_switches = new_switches

        # Check & update opponent_active_pokemon
    #    new_opponent_active = ObservedPokemon_to_list(new_obs["opponent_active_pokemon"], is_active_pokemon=1, is_opponent_team=1)
    #    if new_opponent_active != self.opponent_active_pokemon:
    #        self.opponent_active_pokemon = new_opponent_active

        # Check & update opponent_team
    #    new_opponent_team = get_opponent_team_with_all_infos(new_obs, battle)
    #    if new_opponent_team != self.opponent_team:
    #        self.opponent_team = new_opponent_team

        # Always recompose the output vector
    #    self.output = (
    #        ["side_conditions"] + self.side_conditions +
    #        ["opponent_side_conditions"] + self.opponent_side_conditions +
    #        ["weather"] + self.weather +
    #        ["active_pokemon"] + self.active_pokemon +
    #        ["my_team_switches"] + self.my_team_switches +
    #        ["opponent_active_pokemon"] + self.opponent_active_pokemon +
    #        ["opponent_team"] + self.opponent_team
    #)

Visualizer obs : 
-

In [52]:
from poke_env.player.random_player import RandomPlayer

class MyEnvPlayer(Gen4EnvSinglePlayer):
    def __init__(self, *args, **kwargs):
         super().__init__(*args, **kwargs)
         self._replay_log = True
     
    def embed_battle(self, battle) -> np.ndarray: #renvoie obs
        #if battle.turn == 1 : 
        obs = adapted_obs(battle)
        obs = obs.output
        #else : 
            #obs = obs.update(battle)
            
        return obs

    def calc_reward(self, current_battle,last_battle) -> float:
        # Reward simple : +1 si l'adversaire est KO, -1 si toi tu l'es
        reward = 0
        if last_battle.opponent_active_pokemon and current_battle.opponent_active_pokemon.fainted:
            reward += 1
        if last_battle.active_pokemon and current_battle.active_pokemon.fainted:
            reward -= 1
        return reward

    def describe_embedding(self) -> tuple:
        return (2,), np.float32  # 2 valeurs float (my HP, opp HP)
    
# Adversaire
opponent = RandomPlayer(battle_format="gen4randombattle")

# Ton agent (env Gym-compatible)
env = MyEnvPlayer(battle_format="gen4randombattle", opponent=opponent)

# Lancer un match
obs, info = env.reset()
print(obs)
done = False


['side_conditions', 'opponent_side_conditions', 'weather', 'active_pokemon', 'moltres', 1.0, 'pressure', 'fire_flying', 0, 0, 0, 0, 0, 0, 0, 0, 0.5575397, 0.4186508, 0.38690478, 0.5, 0.37103173, 0.38690478, 'roost', 'airslash', 'fireblast', 'uturn', 'my_team_switches', 'empoleon', 1.0, 'torrent', 'water_steel', 0, 0.51984125, 0.2777778, 0.3670635, 0.43849206, 0.40674603, 0.2777778, 'stealthrock', 'roar', 'icebeam', 'surf', 'hariyama', 1.0, None, 'fighting', 0, 0.7619048, 0.5019841, 0.29960316, 0.23214285, 0.29960316, 0.26587301, 'facade', 'closecombat', 'payback', 'bulletpunch', 'politoed', 1.0, None, 'water', 0, 0.59126985, 0.26785713, 0.35714287, 0.40873015, 0.44444445, 0.34126985, 'hydropump', 'rest', 'icebeam', 'toxic', 'persian', 1.0, None, 'normal', 0, 0.515873, 0.3472222, 0.31349206, 0.32936507, 0.32936507, 0.50595236, 'seedbomb', 'uturn', 'doubleedge', 'bite', 'drapion', 1.0, None, 'poison_dark', 0, 0.5, 0.39087301, 0.4563492, 0.29166666, 0.34126985, 0.40674603, 'crunch', 'taun

In [40]:
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
done = truncated or terminated
#print("reward:", reward)
print(obs)
print("-" * 40)



KeyboardInterrupt: 

In [41]:
env.close()

RuntimeError: The agent is still sending actions. Use this method only when training or evaluation are over.

**On a tout ce qu'on peut avoir :**

pièges posés (pdr, spikes ..)

climat

notre poké actif avec ses pv, talent, ses stats, ses mooves, son nom (species), ses boosts, ses types, ses status

notre team avec ses pv, talent, ses stats, ses mooves, son nom (species), ses types, ses status

poké adverse avec ses pv, talent, ses stats, ses mooves, son nom (species), ses boosts, ses types, ses status

team adverse avec : ses pv, talent, ses stats, ses mooves, son nom (species), ses boosts, ses types, ses status

--> Il nous manquera juste ce qui a été joué par l'adversaire, pour l'instant je passe là-dessus, amsi ça veut dire que notre IA pourra pas s'adapter en fonction du style du joueur

**Embeddings de :**
-
objets

talents

attaques

noms species

status

types


1 : Status
- 
juste un idx pour chaque statut

2 : objets
-


In [43]:
item_to_id = { #Ils ont toujours des objets
    "choiceband": 0,
    "choicescarf": 1,
    "choicespecs": 2,
    "expertbelt": 3,
    "focussash": 4,
    "leftovers": 5,
    "lifeorb": 6,
    "lumberry": 7,
    "powerherb": 8,
    "salacberry": 9,
    "sitrusberry": 10,
    "blacksludge": 11,
    "thickclub": 12,       # os de Ossatueur (Marowak)
    "lightball": 13,       # objet de Pikachu
    "flameplate": 14, 
    "fistplate": 15,
     "earthplate": 16, 
     "stoneplate": 17, 
     "zapplate": 18,
    "meadowplate": 19, 
    "icicleplate": 20, 
    "toxicplate": 21, 
    "mindplate": 22, 
    "insectplate": 23,
    "spookyplate": 24, 
    "dracoplate": 25, 
    "dreadplate": 26, 
    "ironplate": 27, 
    "skyplate": 28,
    "splashplate": 29
}

3 : Talents
-

In [44]:
ability_to_id = {
    "adaptability": 0,
    "aftermath": 1,
    "airlock": 2,
    "anticipation": 3,
    "arenatrap": 4,
    "baddreams": 5,
    "battlearmor": 6,
    "blaze": 7,
    "chlorophyll": 8,
    "clearbody": 9,
    "cloudnine": 10,
    "colorchange": 11,
    "compoundeyes": 12,
    "cutecharm": 13,
    "download": 14,
    "drizzle": 15,
    "drought": 16,
    "dryskin": 17,
    "earlybird": 18,
    "filter": 19,
    "flamebody": 20,
    "flashfire": 21,
    "flowergift": 22,
    "forecast": 23,
    "forewarn": 24,
    "frisk": 25,
    "gluttony": 26,
    "guts": 27,
    "hugepower": 28,
    "hydration": 29,
    "hypercutter": 30,
    "immunity": 31,
    "innerfocus": 32,
    "insomnia": 33,
    "intimidate": 34,
    "ironfist": 35,
    "keeneye": 36,
    "leafguard": 37,
    "levitate": 38,
    "limber": 39,
    "liquidooze": 40,
    "magicguard": 41,
    "magnetpull": 42,
    "marvelscale": 43,
    "minus": 44,
    "moldbreaker": 45,
    "motordrive": 46,
    "multitype": 47,
    "naturalcure": 48,
    "noguard": 49,
    "overgrow": 50,
    "owntempo": 51,
    "pickup": 52,
    "plus": 53,
    "poisonheal": 54,
    "poisonpoint": 55,
    "pressure": 56,
    "purepower": 57,
    "quickfeet": 58,
    "rockhead": 59,
    "roughskin": 60,
    "runaway": 61,
    "sandstream": 62,
    "sandveil": 63,
    "scrappy": 64,
    "serenegrace": 65,
    "shadowtag": 66,
    "shedskin": 67,
    "shellarmor": 68,
    "shielddust": 69,
    "simple": 70,
    "skilllink": 71,
    "slowstart": 72,
    "sniper": 73,
    "snowcloak": 74,
    "snowwarning": 75,
    "solidrock": 76,
    "soundproof": 77,
    "speedboost": 78,
    "static": 79,
    "steadfast": 80,
    "stickyhold": 81,
    "sturdy": 82,
    "suctioncups": 83,
    "superluck": 84,
    "swarm": 85,
    "swiftswim": 86,
    "synchronize": 87,
    "tangledfeet": 88,
    "technician": 89,
    "thickfat": 90,
    "tintedlens": 91,
    "torrent": 92,
    "trace": 93,
    "truant": 94,
    "unburden": 95,
    "vitalspirit": 96,
    "voltabsorb": 97,
    "waterabsorb": 98,
    "waterveil": 99,
    "whitesmoke": 100,
    "wonderguard": 101
}

4 : moves 
-

In [68]:
import pandas as pd

df = pd.read_csv("data/csv/move_to_id.csv")  # ou le chemin local
moves_to_id = dict(zip(df["move"], df["id"]))
moves_to_id["hiddenpower"] = 186

In [69]:
print(moves_to_id)

{'aerialace': 0, 'aeroblast': 1, 'agility': 2, 'airslash': 3, 'aquajet': 4, 'aquatail': 5, 'aromatherapy': 6, 'aurasphere': 7, 'batonpass': 8, 'bellydrum': 9, 'bite': 10, 'blizzard': 11, 'bodyslam': 12, 'bravebird': 13, 'brickbreak': 14, 'bugbite': 15, 'bugbuzz': 16, 'bulkup': 17, 'bulletpunch': 18, 'calmmind': 19, 'chatter': 20, 'closecombat': 21, 'counter': 22, 'crabhammer': 23, 'crosschop': 24, 'crunch': 25, 'curse': 26, 'darkpulse': 27, 'darkvoid': 28, 'destinybond': 29, 'discharge': 30, 'doubleedge': 31, 'dracometeor': 32, 'dragonclaw': 33, 'dragondance': 34, 'dragonpulse': 35, 'drainpunch': 36, 'drillpeck': 37, 'dynamicpunch': 38, 'earthpower': 39, 'earthquake': 40, 'encore': 41, 'energyball': 42, 'eruption': 43, 'explosion': 44, 'extremespeed': 45, 'facade': 46, 'fakeout': 47, 'fireblast': 48, 'firefang': 49, 'firepunch': 50, 'flamethrower': 51, 'flareblitz': 52, 'flashcannon': 53, 'focusblast': 54, 'focuspunch': 55, 'gigaimpact': 56, 'glare': 57, 'grassknot': 58, 'gunkshot': 59

5 : Species
-

In [50]:
df = pd.read_csv("data/csv/species_to_id.csv")  # ou le chemin local
species_to_id = dict(zip(df["species"], df["id"]))

6 : Types
-

In [55]:
# Liste des types Gen 4
types = [
    "bug", "dark", "dragon", "electric", "fighting", "fire", "flying", "ghost",
    "grass", "ground", "ice", "normal", "poison", "psychic", "rock", "steel",
    "water"
]

# Générer les combinaisons mono- et duo-types, sans doublons (ordre alphabétique)
type_combos = set()

for t in types:
    type_combos.add((t,))
    for t2 in types:
        if t != t2:
            type_combos.add(tuple(sorted([t, t2])))

# Convertir en string clés
combo_keys = ["_".join(combo) for combo in sorted(type_combos)]
combo_keys.append("none")

# Générer le dictionnaire
types_to_id = {k: i for i, k in enumerate(combo_keys)}

In [56]:
print(types_to_id)

{'bug': 0, 'bug_dark': 1, 'bug_dragon': 2, 'bug_electric': 3, 'bug_fighting': 4, 'bug_fire': 5, 'bug_flying': 6, 'bug_ghost': 7, 'bug_grass': 8, 'bug_ground': 9, 'bug_ice': 10, 'bug_normal': 11, 'bug_poison': 12, 'bug_psychic': 13, 'bug_rock': 14, 'bug_steel': 15, 'bug_water': 16, 'dark': 17, 'dark_dragon': 18, 'dark_electric': 19, 'dark_fighting': 20, 'dark_fire': 21, 'dark_flying': 22, 'dark_ghost': 23, 'dark_grass': 24, 'dark_ground': 25, 'dark_ice': 26, 'dark_normal': 27, 'dark_poison': 28, 'dark_psychic': 29, 'dark_rock': 30, 'dark_steel': 31, 'dark_water': 32, 'dragon': 33, 'dragon_electric': 34, 'dragon_fighting': 35, 'dragon_fire': 36, 'dragon_flying': 37, 'dragon_ghost': 38, 'dragon_grass': 39, 'dragon_ground': 40, 'dragon_ice': 41, 'dragon_normal': 42, 'dragon_poison': 43, 'dragon_psychic': 44, 'dragon_rock': 45, 'dragon_steel': 46, 'dragon_water': 47, 'electric': 48, 'electric_fighting': 49, 'electric_fire': 50, 'electric_flying': 51, 'electric_ghost': 52, 'electric_grass': 

7 : side_conditions et weather : 
-

In [57]:
weather_to_id = {
    "none": 0,
    "sunnyday": 1,
    "raindance": 2,
    "sandstorm": 3,
    "hail": 4
}
side_condition_to_id = {
    "stealthrock": 1,
    "spikes": 2,
    "toxicspikes": 3,
    "reflect": 4,
    "lightscreen": 5,
    "tailwind": 6
}

In [15]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à deux couche cachée, sortie linéaire, f activation = Relu,
class NNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.out = nn.Linear(128, n_actions)
        #Embedding
        self.item_embedding = nn.Embedding(
        num_embeddings=len(item_to_id),  # nombre total d'objets
        embedding_dim=8                # dimension du vecteur d'embedding
        )
        self.ability_embedding = nn.Embedding(
        num_embeddings=len(ability_to_id),
        embedding_dim= 8
        )
        self.moves_embedding = nn.Embedding(
        num_embeddings=len(moves_to_id),
        embedding_dim= 16
        )
        self.species_embedding = nn.Embedding(
        num_embeddings=len(species_to_id),
        embedding_dim= 32
        )
        self.types_embedding = nn.Embedding(
        num_embeddings=len(types_to_id),
        embedding_dim= 8
        )

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)

In [73]:
#MAJ : tout mettre au format vecteur
def ObservedPokemon_to_list(pokemon,is_active_pokemon,is_opponent_team) -> list: 
    # Nom du Pokémon
    species = getattr(pokemon, "species", "UNKNOWN")
    species_id = species_to_id[species]

    p = Pokemon(gen=4,species= species)


    # Fraction de vie (float entre 0 et 1)
    hp = getattr(pokemon, "current_hp_fraction", 0.0) or 0.0

    # Types (liste de strings ou [])
    types = [p.type_1.name.lower() if p.type_1 else None, p.type_2.name.lower() if p.type_2 else None]
    if p.type_1 and p.type_2 :
        types = sorted([t for t in types])
    type_str = types[0] if types[1] is None else f"{types[0]}_{types[1]}"
    type_id = types_to_id[type_str]

    # Statut (string ou "NONE")
    status = getattr(pokemon, "status", None)
    status_str = status.name if status else "NONE"
    status_to_id = {
     'NONE': 0,
    'par': 1,     # paralysie
    'slp': 2,     # sommeil
    'brn': 3,     # brûlure
    'frz': 4,     # gel
    'psn': 5,     # poison
    'tox': 6      # poison grave
        }
    status_id = status_to_id[status_str] 

    #Boosts
    if(is_active_pokemon == 1) :

        # Boosts (dict ou [0]*7)
        boosts = getattr(pokemon, "boosts", None)
        boosts_list = list(boosts.values()) if boosts else [0] * 7

    # Stats (dict ou [0]*6)
    if is_opponent_team == 0 : #Si c'est pas l'équipe adverse :

        stats = getattr(pokemon, "stats", None)
        stat_list = list(stats.values()) if stats else [0] * 6
        stats_list = [ np.float32(x / 504) for x in stat_list ]

    else : #si c'est l'équipe adverse, on fait autrement
        level = p.level if hasattr(p, "level") and p.level else 80
        hp_stat = floor(((2 * p.base_stats["hp"] + 31 + 84 // 4) * level) / 100) + level + 10
        if hp == None : 
            hp = 1.0
            #On rajoute un quasi-normalize, pour aller autour de 1 sans que ça soit trop petit, donc on prend pas l'exemple leuphorie 714 stats en PV avec lvl 100, nature, EVs
        atk = floor((2 * p.base_stats["atk"] + 31 + 84 // 4) * level / 100 + 5) 
        def_ = floor((2 * p.base_stats["def"] + 31 + 84 // 4) * level / 100 + 5) 
        spa = floor((2 * p.base_stats["spa"] + 31 + 84 // 4) * level / 100 + 5) 
        spd = floor((2 * p.base_stats["spd"] + 31 + 84 // 4) * level / 100 + 5) 
        spe = floor((2 * p.base_stats["spe"] + 31 + 84 // 4) * level / 100 + 5) 
        
        stats_list = [hp_stat,np.float32(atk/504),np.float32(def_/504),np.float32(spa/504),np.float32(spd/504),np.float32(spe/504)]
    #moovepool
    moves_dict = getattr(pokemon, "moves", None)
    move_names = list(moves_dict.keys()) if moves_dict else []

    # Pour avoir toujours 4 valeurs :
    while len(move_names) < 4:
        move_names.append("None")

    move_names_id = [moves_to_id[x.lower().replace(" ", "")] for x in move_names if x and x.lower() != "none"]
    
    ability = p.ability
    ability_id = ability_to_id[ability] if ability else -1 # Pas sûr du -1
    
    if(is_active_pokemon == 1) :

        return [
            species_id,
            hp,
            ability_id,
            type_id,
            status_id,
            *boosts_list,
            *stats_list,
            *move_names_id[:4]
        ]
    else : 
        return [
            species_id,
            hp,
            ability_id,
            type_id,
            status_id,
            *stats_list,
            *move_names_id[:4]
        ]

In [75]:
#on redéfinit obs, pour cette fois-ci avoir tout au format vecteur, et plus rien en str
class adapted_obs :
    def __init__(self,battle) :
        self.dict_observation = battle.current_observation.__dict__ 
        #et stocker l'état précédent ? Pour l'instant on passe
        self.side_conditions = list(self.dict_observation["side_conditions"].values())
        self.side_conditions = np.array([(side_condition_to_id(x)/6) for x in self.side_conditions])

        self.opponent_side_conditions = list(self.dict_observation["opponent_side_conditions"].values())
        self.opponent_side_conditions = np.array([(side_condition_to_id(x)/6) for x in self.opponent_side_conditions])

        self.weather = list(self.dict_observation["weather"].values())
        self.weather = np.array([(weather_to_id(x)/5) for x in self.weather])

        self.active_pokemon =np.array( ObservedPokemon_to_list(self.dict_observation["active_pokemon"],is_active_pokemon=1,is_opponent_team=0))
        self.my_team_switches = np.array(get_my_switches_with_all_infos(battle.available_switches,self.dict_observation))
        self.opponent_active_pokemon = np.array(ObservedPokemon_to_list(self.dict_observation["opponent_active_pokemon"],is_active_pokemon=1,is_opponent_team=1)) #à 1 car on veut savoir si il est boosté
        
        #Equipe adverse sans le pokemon actif 
        self.opponent_team = np.array(get_opponent_team_with_all_infos(self.dict_observation,battle))
        #Fin équipe adverse 
        self.output =  self.side_conditions + self.opponent_side_conditions  + self.weather  + self.active_pokemon + self.my_team_switches + self.opponent_active_pokemon + self.opponent_team


In [76]:
from poke_env.player.random_player import RandomPlayer

class MyEnvPlayer(Gen4EnvSinglePlayer):
    def __init__(self, *args, **kwargs):
         super().__init__(*args, **kwargs)
         self._replay_log = True
     
    def embed_battle(self, battle) -> np.ndarray: #renvoie obs
        #if battle.turn == 1 : 
        obs = adapted_obs(battle)
        obs = obs.output
        #else : 
            #obs = obs.update(battle)
            
        return obs

    def calc_reward(self, current_battle,last_battle) -> float:
        # Reward simple : +1 si l'adversaire est KO, -1 si toi tu l'es
        reward = 0
        if last_battle.opponent_active_pokemon and current_battle.opponent_active_pokemon.fainted:
            reward += 1
        if last_battle.active_pokemon and current_battle.active_pokemon.fainted:
            reward -= 1
        return reward

    def describe_embedding(self) -> tuple:
        return (2,), np.float32  # 2 valeurs float (my HP, opp HP)
    
# Adversaire
opponent = RandomPlayer(battle_format="gen4randombattle")

# Ton agent (env Gym-compatible)
env = MyEnvPlayer(battle_format="gen4randombattle", opponent=opponent)

# Lancer un match
obs, info = env.reset()
print(obs)
done = False

[39, 1.0, 7, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0.53968257, 0.49603173, 0.32936507, 0.46230158, 0.32936507, 0.36309522, 155, 149, 48, 165, 272, 1.0, -1, 132, 0, 0.5714286, 0.5297619, 0.3452381, 0.3452381, 0.3452381, 0.2797619, 158, 25, 46, 21, 281, 1.0, -1, 0, 0, 0.5734127, 0.29563493, 0.32738096, 0.29563493, 0.4047619, 0.44444445, 8, 41, 160, 16, 230, 1.0, -1, 15, 0, 0.45833334, 0.48015872, 0.3888889, 0.25396827, 0.32936507, 0.28373015, 155, 18, 158, 15, 157, 1.0, 20, 75, 0, 0.515873, 0.40674603, 0.31547618, 0.50595236, 0.40674603, 0.3670635, 48, 161, 163, 40, 246, 1.0, -1, 132, 0, 0.75793654, 0.43452382, 0.2936508, 0.2936508, 0.43452382, 0.1845238, 12, 26, 141, 122, 202, 1.0, -1, 62, 0, 0, 0, 0, 0, 0, 0, 0, 292, 0.5297619, 0.35119048, 0.35119048, 0.39087301, 0.49007937, 202, 1.0, -1, 62, 0, 292, 0.5297619, 0.35119048, 0.35119048, 0.39087301, 0.49007937]


Agent : 
-

In [14]:
############## DEFINIR L'EMBEDDING ##################

# Initialiser GenData pour la génération souhaitée (par exemple, génération 4)
gen_data = GenData.from_gen(4)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class Player4Training(Gen4EnvSinglePlayer): #Classe pour entrainer le modèle 4. 
    def __init__(self,model_path=None, **kwargs):
        super().__init__(**kwargs)
        if model_path is not None :
            #self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
            print(f"📥 Modèle chargé depuis {model_path}")

        self.action_space = Discrete(9)  # ✅ attribut classique
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle)  :
        obs = adapted_obs(battle)
        obs = obs.output
        return obs

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 72 +         # my team types
            [0] * 72           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 72 +         # my team types
            [1] * 72           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        order = super().action_to_move(action, battle)
        return order

In [16]:
class AdversarialTrainPlayer(RandomPlayer): # Adversaire de Player5 lors de l'entrainement

    def __init__(self, model_path="Players/embedding_2", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 17)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)
    
    #action_to_move suppr
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action
    
    def choose_move(self, battle):
        obs = self.embed_battle(battle)
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model.q_net(obs_tensor)
            action = int(torch.argmax(q_values).item())

        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            return self.choose_random_move(battle)

Train
-


In [17]:
from stable_baselines3.common.vec_env import DummyVecEnv

# Réinstancier l'env
train_env_raw = Player4Training(
    battle_format="gen4randombattle",
    opponent=RandomPlayer(battle_format="gen4randombattle"),
    start_challenging=True
)

train_env = DummyVecEnv([lambda: train_env_raw])  # Redéfini bien après reset

In [18]:
env = train_env

model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=1,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[128, 64])
)

NameError: name 'DQN' is not defined

2025-05-09 23:21:40,918 - Player4Training 1 - ERROR - Unhandled exception raised while handling message:
>battle-gen4randombattle-36492
|player|p2|RandomPlayer 3|102|
|teamsize|p1|6
|teamsize|p2|6
|gen|4
|tier|[Gen 4] Random Battle
|rule|Sleep Clause Mod: Limit one foe put to sleep
|rule|HP Percentage Mod: HP is shown in percentages
|
|t:|1746825700
|start
|switch|p1a: Delcatty|Delcatty, M|302/302
|switch|p2a: Umbreon|Umbreon, L85, F|100/100
|turn|1
Traceback (most recent call last):
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/ps_client/ps_client.py", line 136, in _handle_message
    await self._handle_battle_message(split_messages)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/my_showdown_ai_git/showdown_4/lib/python3.11/site-packages/poke_env/player/player.py", line 361, in _handle_b

In [ ]:
#steps = 1000000
#callback = CustomTQDMCallback(total_timesteps = steps, check_freq=1000, verbose=0)
#model.learn(
#    total_timesteps=steps,
#    callback=callback
#)
#model.save("Players/player_4_dynamic")

Eval
-